<img src="../imgs/Adevinta-ULPGC-logo.jpg" width="530px" align="right">

# **Modelos del lenguaje basados en redes neuronales artificiales**

## **Representación vectorial del texto**


>Dime con quién andas y te diré quién eres.


La representación vectorial del texto usando redes neuronales han revolucionado el campo del procesamiento de lenguaje natural en la última década. Estas representaciones, comúnmente conocidas como "embeddings", capturan relaciones semánticas y sintácticas entre palabras, frases o incluso documentos completos en un espacio de menor dimensión. Veamos los modelos más destacados.

### **Word2Vec (Mikolov et al., 2013)**

Word2Vec es uno de los modelos más populares para aprender representaciones vectoriales de palabras, y su introducción marcó un antes y un después en la forma en que se abordan muchas tareas de NLP. Fue propuesto por Mikolov et al. en 2013. Se basa en la idea de que el significado o función de una palabra puede inferirse por el contexto en el que aparece frecuentemente. Por lo tanto, palabras con contextos similares tendrán representaciones vectoriales similares. Word2Vec tiene dos arquitecturas principales: Skip-Gram y CBOW (Continuous Bag of Words).


#### **Skip-Gram**

Dada una palabra objetivo, el modelo Skip-Gram trata de predecir las palabras de su contexto cercano. Por ejemplo, en la frase "El color azul me gusta más que el rojo", si "azul" es nuestra palabra objetivo y utilizamos un tamaño de ventana de contexto de 2, querríamos predecir las palabras "El", "color", "me" y "gusta" a partir de "azul". Si en otra frase del dataset apareciera la frase "El color rojo me gusta más que el azul", el modelo aprendería que "azul" y "rojo" son palabras que podrían compartir contextos similares y, por tanto, sus vectores tenderían a tener una representación similar para poder predecir correctamente las palabras de su contexto.

<p align="center">
<img src="./imgs/Skip-Gram.svg">
</p>

#### **CBOW**

Dado un contexto, CBOW trata de predecir la palabra objetivo o central. Por ejemplo, en la frase "El color azul me gusta más que el rojo", si tomamos un tamaño de ventana de contexto de 2 alrededor de la palabra "me", las palabras de contexto serían "color", "azul", "más" y "que", y CBOW intentaría predecir "rojo" a partir de estas palabras. A diferencia de Skip-Gram, en CBOW todas las palabras de contexto se suman o promedian, de ahí el nombre "Bag of Words" ya que no se considera el orden de las palabras.

<p align="center">
<img src="./imgs/CBOW.svg">
</p>



##### **Código**
https://towardsdatascience.com/word2vec-with-pytorch-implementing-original-paper-2cd7040120b0

## **GloVe**

https://nlp.stanford.edu/pubs/glove.pdf

GloVe, que significa "Global Vectors for Word Representation", es un método para la obtención de representaciones vectoriales (embeddings) de palabras. Fue desarrollado por investigadores de la Universidad de Stanford en 2014. A diferencia de otros métodos, como Word2Vec, que se basan en contextos locales de palabras, GloVe construye sus embeddings basándose en estadísticas globales de co-ocurrencia de palabras en un corpus.

Los elementos esenciales del modelo GloVe son:

#### **Matriz de co-ocurrencia**

El primer paso para crear embeddings con GloVe es construir una matriz de co-ocurrencia $ X $ a partir de un corpus de texto. Si consideramos $ X_{ij} $, este valor representa cuántas veces la palabra $ j $ aparece en el contexto de la palabra $ i $. El "contexto" puede definirse de muchas maneras, pero generalmente se refiere a un conjunto fijo de palabras que aparecen antes o después de la palabra objetivo.

#### **Idea Principal** 

GloVe busca factorizar la matriz de co-ocurrencia en dos matrices de menor dimensión, cuyo producto aproximado (y algunas operaciones adicionales) es la matriz original. Sin embargo, en lugar de factorizar la matriz de co-ocurrencia directamente, GloVe opera sobre la función logarítmica de la matriz.


#### **Objetivo**

El objetivo de GloVe es minimizar la siguiente función de pérdida:

$$ J = \sum_{i,j=1}^{V} f(X_{ij}) \left( w_i^T \tilde{w}_j + b_i + \tilde{b}_j - \log X_{ij} \right)^2 $$



Donde:
- $ V $ es el tamaño del vocabulario.
- $ w_i $ y $ \tilde{w}_j $ son los vectores de embedding de las palabras $ i $ y $ j $, respectivamente.
- $ b_i $ y $ \tilde{b}_j $ son términos de sesgo para las palabras $ i $ y $ j $.
- $ f $ es una función de ponderación. 

----


#### **Intuición**

Asumimos que en la matriz de co-ocurrencia, cada par de palabras (filas) $ X_i $ y $ X_j $ son semánticamente similares si sus componentes $ X_{ik} $ y $ X_{jk} $ son similares para todo $k$. Como cada fila tiene una longitud igual al tamaño del vocabulario, sería conveniente que cada fila (palabra) fuera un vector de embedding. Por lo tanto, si dos filas son similares, sus vectores de embedding también deberían serlo.

Así que si hemos minimizado la función de pérdida: $J = \sum_{i,j=1}^{V} \left( w_i w_j^T - X_{ij} \right)^2$, entonces cada elemento de la matriz de co-ocurrencia $X_{ij}$ debería ser aproximadamente igual a $w_i w_j^T$.

Vemos que lo que hemos conseguido es la factorización de la matriz de co-ocurrencia en dos matrices de menor dimensión, cuyo producto aproximado es la matriz original.

<div align="center">
    <img src="./imgs/GLOVE_01.svg" width="50%">
</div>

Por tanto, si $X_{ik}$ es aproximadamente $\omega_i \cdot \omega_k$ y $X_{jk}$ es aproximadamente $\omega_j \cdot \omega_k$ para todo $k$, entonces $\omega_i$ y $\omega_j$ deberían ser similares en todos sus componentes.



----

La función de ponderación $ f(x) $ en el modelo GloVe es una función que da menos peso a las co-ocurrencias menos frecuentes y más peso a las más frecuentes, pero hasta cierto punto. La idea es prevenir que las co-ocurrencias de alta frecuencia dominen el entrenamiento.

La función $ f(x) $ que se introduce en el trabajo original de GloVe es la siguiente:

$$
f(x) = 
\begin{cases} 
(x / x_{\text{max}})^{\alpha} & \text{si } x < x_{\text{max}} \\
1 & \text{de lo contrario}
\end{cases}
$$

Donde:
- $ x $ es el valor de co-ocurrencia de la matriz $ X $.
- $ x_{\text{max}} $ es un umbral definido para ponderar las co-ocurrencias. En el trabajo original, este valor se elige como 100.
- $ \alpha $ es un exponente que determina cuán rápido decrece la función. En el trabajo original de GloVe, $ \alpha $ se elige como 0.75.

Esta función asegura que para co-ocurrencias bajas, $ f(x) $ es proporcional a $ x $, mientras que para co-ocurrencias altas (aquellas mayores que $ x_{\text{max}} $), la función se aplana y simplemente se toma como 1. El propósito de esto es dar más relevancia a las co-ocurrencias raras durante el entrenamiento, pero sin ignorar completamente las frecuentes.

<p align="center">
<img src="./imgs/weighting_function.png" width="30%">
</p>


#### **Resultados**

Una vez entrenado el modelo, se pueden obtener representaciones vectoriales para cada palabra. A menudo, el vector de embedding final para una palabra se toma como la suma de su vector $ w $ y su vector $ \tilde{w} $.

#### **Comparación con otros métodos**

A diferencia de Word2Vec, que usa contextos locales y se basa en la predicción (ya sea usando el contexto para predecir una palabra o viceversa), GloVe se basa en estadísticas de co-ocurrencia a nivel global. Sin embargo, en la práctica, ambos métodos producen representaciones de palabras de alta calidad que capturan similitudes semánticas y relaciones sintácticas entre palabras.

#### **Uso**

Una vez entrenado, un modelo GloVe, al igual que Skip-Gram o CBOW, permite que las palabras con significados similares se mapeen a puntos cercanos en el espacio vectorial, facilitando así tareas como la clasificación de texto, la analogía de palabras y la agrupación semántica, entre otras.

### **Intuiciones**

#### **¿Por qué se usa la función logarítmica de la matriz de co-ocurrencia?**

En cualquier corpus de texto, las co-ocurrencias varían drásticamente en magnitud. Algunos pares de palabras pueden co-ocurrir miles de veces, mientras que otros solo unas pocas veces. El logaritmo tiene la propiedad de suavizar estas diferencias, es decir, reduce la disparidad entre las co-ocurrencias altas y bajas. Esto hace que el modelo no esté excesivamente influenciado por pares de palabras que co-ocurren con mucha frecuencia.



<img src="./imgs/torchtext_logo.png" width="25%">

## **PyTorch: TorchText**

**TorchText** es una biblioteca complementaria a PyTorch, diseñada específicamente para las tareas de procesamiento de lenguaje natural (NLP). La biblioteca incluye herramientas para cargar, procesar y trabajar con conjuntos de datos de texto, así como para construir vocabularios y manipular secuencias de texto.

https://pytorch.org/text/stable/index.html

In [40]:
# Read the 10 first lines of WikiText2 dataset in TorchText


import torch
from torchtext import datasets

# Load the dataset
train_iter = datasets.WikiText2(split=('train'))

# Print the first 10 lines
for i, line in enumerate(train_iter):
    if i < 10:
        print(line)
        print("-" * 50)
    else:
        break

# print(train_iter[100])  # Error because the dataset is not a map-style dataset

# Convert the dataset to map-style dataset
from torchtext.data import to_map_style_dataset

train_iter = to_map_style_dataset(train_iter)

print(train_iter[4])
print(train_iter[1001])

 

--------------------------------------------------
 = Valkyria Chronicles III = 

--------------------------------------------------
 

--------------------------------------------------
 Senjō no Valkyria 3 : <unk> Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . <unk> the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " <unk> Raven " . 

--------------------------------------------------
 The game began development in 2010 , carrying over a large port

---

### Ejercicio 1

Desarrolla un script para generar los datasets de entrenamiento adecuados para suministrarlos a lo modelos Skip-Gram y CBOW.

---

---

### Ejercicio 2

Desarrolla un script para generar la matriz de co-ocurrencia a partir del dataset wikitext-2.

---